## Exploratory Analysis

That's it for some preliminary cleaning. Don't worry, there will be more. Let's start to look in a bit more detail at the data, though. In this section, we're going to start to write some code that's typical for day-to-day data cleaning tasks.

In [1]:
from load_data import dta

We can use `info` to get some high-level information about the data.

In [2]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 68091 to 2009279
Data columns (total 15 columns):
address            25000 non-null object
aka_name           24731 non-null object
city               24978 non-null object
dba_name           25000 non-null object
facility_type      24565 non-null category
inspection_date    25000 non-null datetime64[ns]
inspection_type    25000 non-null category
latitude           24822 non-null float64
license_           24999 non-null float64
longitude          24822 non-null float64
results            25000 non-null category
risk               24991 non-null category
state              24995 non-null object
violations         19538 non-null object
zip                24991 non-null object
dtypes: category(4), datetime64[ns](1), float64(3), object(7)
memory usage: 2.4+ MB


And `describe` goes into a bit more detail for the *numeric* types, of which we don't have many here.

In [3]:
dta.describe()

,latitude,license_,longitude
count,24822.000000,2.499900e+04,24822.000000
mean,41.879724,1.566116e+06,-87.676341
std,0.081331,8.968458e+05,0.058746
min,41.644670,0.000000e+00,-87.914428
25%,41.830500,1.167576e+06,-87.708082
50%,41.891055,1.959049e+06,-87.665706
75%,41.939776,2.215768e+06,-87.634955
max,42.021064,8.700606e+06,-87.525094


We could do the same for the categorical types.

In [4]:
dta.select_dtypes(['category']).describe()

,facility_type,inspection_type,results,risk
count,24565,25000,25000,24991
unique,244,45,7,4
top,Restaurant,Canvass,Pass,Risk 1 (High)
freq,16666,12305,15827,17885


### GroupBy

Now, let's ask the most obvious question. Which are the best and the worst restaurants? We'll want to use pandas `GroupBy` functionality to implement the `split-apply-combine` pattern.

The idea here is that we **split** the data by some key or set of keys then **apply** a function to each group and then **combine** the outputs back into a single DataFrame.

First, let's see how many result categories there are. We can use `value_counts` to answer this question. 

In [6]:
import pandas as pd
with pd.option_context("max.rows", 10):
    print(dta.results.value_counts())

Pass                    15827
Fail                     4876
Pass w/ Conditions       2396
Out of Business          1088
No Entry                  547
Not Ready                 263
Business Not Located        3
Name: results, dtype: int64


Ok, let's group on the inspection `results` column and see who the best and worst are.

When we call the `groupby` method we get back a `DataFrameGroupBy` object.

In [7]:
grouper = dta.groupby(dta.results)

In [8]:
grouper

You can access the variables on this object, the same as a DataFrame, and any code called will execute within the groups.

In [9]:
result = grouper.dba_name.value_counts()

The result is a Series with a `MultiIndex`.

In [10]:
result

results               dba_name                                          
Business Not Located  CITGO SUPER WASH & GAS                                 1
                      KAIYO                                                  1
                      NANNY'S WAFFALS HOUSE                                  1
Fail                  SUBWAY                                                41
                      DUNKIN DONUTS                                         33
                      MCDONALD'S                                            23
                      MCDONALDS                                              9
                      PIZZA HUT                                              9
                      7-ELEVEN                                               7
                      AU BON PAIN                                            7
                      DOMINO'S PIZZA                                         7
                      HAROLD'S CHICKEN SHACK              

In [ ]:
result.index.names

We can index on the first element in a `MultiIndex` using square brackets and then use `sort_values` to find those restaurants that had a result of Fail the most.

In [ ]:
with pd.option_context('max.rows', 15):
    print(result["Fail"].sort_values(ascending=False))

Take a closer look above. Looks like we have some more data cleaning to do.

In [ ]:
with pd.option_context('max.rows', 15):
    print(result["Pass"].sort_values(ascending=False))

This is probably not the right way to think about this since there are many more Subways than local establishments.

We could instead look at the ratio of Fail to Pass, though, of course, this isn't perfect either. 

Sometimes, it's not *always* obvious how to go about computing things that you want to compute. The `get_group` method allows you to pull out one of the split DataFrames and try your apply function on it.

## Exercise

GroupBy the `dba_name`. Use `get_group` to pull out the "MCDONALD'S" group. Write a function that calculates the relative number of Fail to Pass for this group. Run this function on the McDonald's group.

In [ ]:
# Type your solution here

In [ ]:
# %load solutions/get_group.py
grouper = dta.groupby(dta.dba_name)
mcd = grouper.get_group("MCDONALD'S")


def relative_results(df):
    values = df.value_counts()
    return values['Fail'] / values['Pass']


relative_results(mcd.results)

# And we see McDonald's failed 50% as many inspections as it Passed.


We can run this on everything, but it's going to be a little slow. Let's look at another way to approach this problem.

Here group by *both* the inspection results and the DBA name. Then we ask for the `size` of each one of these groups.

In [12]:
result = dta.groupby((dta.results, dta.dba_name)).size()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [13]:
result

results               dba_name                                          
Business Not Located  CITGO SUPER WASH & GAS                                1
                      KAIYO                                                 1
                      NANNY'S WAFFALS HOUSE                                 1
Fail                  #1 CHOP SUEY                                          2
                      #1 WOK N ROLL                                         1
                      1 N WACKER KITCHEN AND BAR                            1
                      111 TH FOOD & CELLULAR, INC                           1
                      111TH AND RACINE MARATHON, INC.                       1
                      111TH STREET OTB                                      1
                      12 WEST ELM                                           1
                      123 MINI MART                                         1
                      13 Pins Tapas & Grill                          

We can use the `div` method to divide these for us. As you can see the indices don't line up, but we don't have to worry about it. Pandas take care of index alignment for us.

In [14]:
result["Fail"]

dba_name
#1 CHOP SUEY                                     2
#1 WOK N ROLL                                    1
1 N WACKER KITCHEN AND BAR                       1
111 TH FOOD & CELLULAR, INC                      1
111TH AND RACINE MARATHON, INC.                  1
111TH STREET OTB                                 1
12 WEST ELM                                      1
123 MINI MART                                    1
13 Pins Tapas & Grill                            1
1492 TAPAS                                       1
16TH & MILLARD FOOD                              1
16TH ST FOOD MART                                1
18TH STREET FOOD MART                            1
1914 CLUB                                        2
194  RIB  JOYNT                                  1
1ST FRUITS ACADEMY INC                           1
2 ASIAN BROTHERS                                 1
2002 DONUTS INC                                  1
2012 FOOD MART INC                               1
24 FOODS              

In [ ]:
result["Pass"]

In [15]:
ratio = result["Fail"].div(result["Pass"])
ratio.sort_values(ascending=False, inplace=True)
ratio

dba_name
JACK'S ON HALSTED                                    4.000000
NORTHSTAR COFFEE                                     4.000000
O'LEARY'S PUBLIC HOUSE                               3.000000
KING GYROS                                           3.000000
PILSEN ELEMENTARY                                    3.000000
SOUTH SHORE INTERNATIONAL COLLEGE PREP               3.000000
ROSELAND FOOD & LIQUOR                               3.000000
FERNANDO'S RESTAURANT                                3.000000
MELROSE RESTAURANT                                   3.000000
FAIRPLAY FOODS                                       3.000000
PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO    3.000000
TAMALLI SPACE CHARROS LLC                            3.000000
THE GODDESS AND GROCER                               3.000000
TABO SUSHI                                           3.000000
SUBWAY 3634                                          3.000000
EL FARO RESTAURANT                                   3.000000

We have a lot of `NaN`s in the results from division-by-zero. We can drop those with a call to `dropna`. Also note that pandas lets you decide whether to treat `inf` as an NA.

In [ ]:
with pd.option_context("use_inf_as_null", True,
                       "max.rows", 15):
    print(ratio.dropna())

We might still not be wholly satisfied with our rules around comparisons here. First, we're looking at restaurant names not particular establishments. What does the distribution of inspection visits for establishments look like.

In [ ]:
with pd.option_context("max.rows", 10):
    grouper = dta.groupby((dta.address, dta.dba_name))
    print(grouper.size().describe())

Ok, let's make things a little more challenging. Let's see what the Fail:Pass ratio is for restaurants with at least 3 visits that involved a high risk level. 


Now we're starting to get into some much more powerful pandas constructs.

In [ ]:
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))

visited

Let's unpack this. The first thing to note is how this code is organized. Each one of these methods return a pandas data structure on which we call the next method. This is called **method chaining**. We use the same trick seen above to split strings across lines to split several method calls by including the code between `()`.

Next, we see several new methods. The first is **query**. When subsetting a DataFrame we have a few options. As we save above, we can index a DataFrame using integers. Likewise, we could pass an object of booleans as well.

In [ ]:
dta.risk == "Risk 1 (High)"

In [ ]:
dta.loc[dta.risk == "Risk 1 (High)"]

Always using indexing can be verbose, however. You may need compound statements, for example.

In [ ]:
dta.loc[(dta.risk == "Risk 1 (High)") | (dta.risk == "Risk 1 (Medium)")].head()

Instead, by using query we could write the following, which is slightly easier.

In [ ]:
dta.query("(risk == 'Risk 1 (High)') | (risk == 'Risk 1 (Medium)')").head()

## Exercise

Use `query` to find the visits that are to restaurants and that are complaint re-inspections.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/query.py

```python
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')  # size returns a nameless series
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))
```


The next new method is the **rename** method. We use this to rename the unnamed Series returned by `size`.

Finally, we filter on restaurants with 4 or more total visits.

The final piece is computing the Fail:Pass ratio of these restaurants. To do this, we need to take the output we've created `visited` and line our original data up with these addresses and DBA names.

In [ ]:
visited.head()

We can do this by using the **merge** method. Merge allows us to make two pandas DataFrames into a single DataFrame. By default, the `merge` method will join together two DataFrames on common columns, using an inner join method (a set intersection).

In [ ]:
merged_visits = visited.merge(dta)
merged_visits.head()

Now, we need to redo the analysis we started above for McDonald's. We take these merged DataFrames, group them by the inspection results, the address, and the DBA name and ask for the size of each group.

In [ ]:
(merged_visits
 .groupby(('results', 'address', 'dba_name'))
 .size()).head()

## Exercise

Take this result and **pipe** it (using the `pipe` method) to a function that computes Fail/Pass. Make sure your result does not have any missing values, and sort it such that those with the highest Fail/Pass ratios are highest.

The **pipe** method allows for including user-defined functions in method chains. It takes the output of the thing on the left, and passes it to the thing on the right.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/pipe_results.py

## Exercise

Now, take everything that we've done above, from `dta` to this final result, and put it together into a single method chain.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/complete_chain.py

Often, I'll try to place a method chain to get the data ready for more exploratory work at the top of a notebook, so I can proceed with any analyses.

Now, let's go back to our original data and add in the unstacked violations to the information that's unchanging. Recall from the previous notebook that we unstack the violations as follows. There are two new things to note here though. We add in a `to_frame` method to turn the unstacked Series into a DataFrame, and we `rename` the unnamed column in the resulting DataFrame back to `violations`.

In [ ]:
(dta.violations
 .str.split("|", expand=True)
 .unstack()
 .dropna()
 .reset_index(level=0, drop=True)
 .str.strip()
 .rename('violations')
 .to_frame())

First, we need to drop the violations from the original DataFrame, then we need to merge it with the unstacked violations Series that we created before. You can use `drop` to do this.

In [ ]:
dta.drop(["violations"], axis='columns').head()

## Exercise

Drop the original violations from `dta`, and **join** this to the unstacked violations as computed above. You'll probably want to use a **right join**.

We use **join** here rather than **merge**. Join uses merge under the hood but conveniently allows us to join on the indices of the two DataFrames by default. One other difference is that join uses an inner merge by default, but that's not what we want here. Since we drop the null violations on the right-hand side DataFrame, we want to do a right join. 

In [ ]:
# Type your solution here

In [ ]:
%load solutions/join_violations.py

Now that we have a relatively clean DataFrame, let's ask a few more questions. 

First, how many unique violations do we have?

In [ ]:
dta.shape

In [ ]:
dta.violations.head()

In [ ]:
dta.violations.unique().shape

Is this true? Do we really think there are this many violation numbers? Probably not. We can use the `str` accessor and some more munging to answer this. Here we pass a **regular expression** to `str.extract`. Extract expects a *capture group*, indicated by `()`. The regular expression `(\d+\)(?=\.)` means capture 1 or more (`+`) digits (`\d`) that is followed by (`(?=)`) a period `\.`. We escape the period because a plain `.` is a wildcard for any character.

In [ ]:
(dta.violations
 .str.extract("(\d+)(?=\.)", expand=False)
 .astype(int))

So how many unique violations do we have?

In [ ]:
np.sort(
    dta.violations
        .str.extract("(\d+)(?=\.)", expand=False)
        .astype(int)
        .unique()
).shape

Second, can we figure out how many times an establishment previously failed an inspection (within the sample we have)? How might we approach this? 

First, we want to restrict the data to just a single row for each inspection. Since we merged everything with the unstacked violations above, we'll need to use `drop_duplicates` to do this.

In [ ]:
visits = dta.drop_duplicates(["address", "dba_name", "inspection_date"])

We're going to rely on some pandas time-series functionality to do this, so we will need to ensure that the inspection dates are sorted within each group. GroupBy will preserve this.

In [ ]:
visits = visits.sort_values(["address", "dba_name", "inspection_date"])

In [ ]:
grouper = visits.groupby((visits.address, visits.dba_name))

Ok, we might ask, "now what?" Remember the trick to pull out groups? Let's use it to work with something we can think about.

In [ ]:
group_key = list(grouper.groups.keys())[0]

In [ ]:
group_key

In [ ]:
group = grouper.get_group(group_key)

In [ ]:
group

In [ ]:
group[['inspection_date', 'results']]

Since, we need this to be backwards looking, we will **shift** the data by one visit. Shifting will move the data around by either a number of periods or a frequency. In this case, we use a number of periods and shift forward by 1 period.

In [ ]:
group.shift(1)[['inspection_date', 'results']]

If we take the cumulative sum of this, we'll have an accurate picture of previous failures.

In [ ]:
(group.shift(1).results == 'Fail').cumsum()

In [ ]:
visit_num = grouper.apply(lambda df: (df.shift(1).results == 'Fail').cumsum())

In [ ]:
visit_num.head(n=15)

In [ ]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

In [ ]:
dta.head()

In [ ]:
visit_num.head()

In [ ]:
visit_num.reset_index(level=[0, 1], drop=True).head()

In [ ]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

In [ ]:
dta.join((visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
)))